# MTTF

In [3]:
# Copyright (c) 2025
# All rights reserved.
#
# Author: Ahmad Fauzi Ridwan
# Created: July 2025

# Instalasi ipywidgets
!pip install ipywidgets --quiet

# Import library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import expon, norm, lognorm, weibull_min, ks_1samp
from scipy.special import gamma
import ipywidgets as widgets
from IPython.display import display, clear_output

# Input widget
input_ttf = widgets.Textarea(
    value='3107.5\n7677.25\n925\n2751\n8567.67\n7156\n4390.5\n5422.5',
    placeholder='Paste the TTF data here (without header, 1 data per line)',
    description='Data TTF:',
    layout=widgets.Layout(width='100%', height='200px'),
    style={'description_width': 'initial'}
)

run_button = widgets.Button(
    description='🔍 Calculate MTTF & Reliability',
    button_style='success'
)

output = widgets.Output()

# Function when button is pressed
def on_button_click(b):
    with output:
        clear_output()
        try:
            # Retrieve and parse data
            ttf_lines = input_ttf.value.strip().split('\n')
            ttf_data = np.array([float(x.strip()) for x in ttf_lines if x.strip() != ''])

            if len(ttf_data) < 3:
                print("⚠️ Enter at least 3 TTF data.")
                return

            # Fit distribution
            distribusi = {
                "Exponential": expon.fit(ttf_data, floc=0),
                "Weibull": weibull_min.fit(ttf_data, floc=0),
                "Lognormal": lognorm.fit(ttf_data, floc=0),
                "Normal": norm.fit(ttf_data)
            }

            # KS Test
            ks_results = {}
            for name, params in distribusi.items():
                dist = {
                    "Exponential": expon,
                    "Weibull": weibull_min,
                    "Lognormal": lognorm,
                    "Normal": norm
                }[name]
                ks_stat, _ = ks_1samp(ttf_data, lambda x: dist.cdf(x, *params))
                ks_results[name] = (ks_stat, params)

            # Best distribution (smallest KS)
            best_dist = min(ks_results, key=lambda k: ks_results[k][0])
            best_params = ks_results[best_dist][1]

            print(f"✅ Best distribution (based on KS-Statistic): {best_dist}")

            # Calculate MTTF & Reliability
            if best_dist == "Weibull":
                shape, loc, scale = best_params
                mttf = scale * gamma(1 + 1/shape)
                R = np.exp(-(mttf / scale) ** shape)
                pdf_func = lambda t: weibull_min.pdf(t, shape, loc, scale)
                rel_func = lambda t: np.exp(-(t / scale) ** shape)
            elif best_dist == "Lognormal":
                shape, loc, scale = best_params
                mu = np.log(scale)
                sigma = shape
                mttf = np.exp(mu + (sigma**2)/2)
                z = (np.log(mttf) - mu) / sigma
                R = 1 - norm.cdf(z)
                pdf_func = lambda t: lognorm.pdf(t, sigma, loc, scale)
                rel_func = lambda t: 1 - lognorm.cdf(t, sigma, loc, scale)
            elif best_dist == "Normal":
                mu, sigma = best_params
                mttf = mu
                R = 0.5
                pdf_func = lambda t: norm.pdf(t, mu, sigma)
                rel_func = lambda t: 1 - norm.cdf(t, mu, sigma)
            elif best_dist == "Exponential":
                loc, scale = best_params
                mttf = scale
                R = np.exp(-1)
                pdf_func = lambda t: expon.pdf(t, loc, scale)
                rel_func = lambda t: np.exp(-t / scale)

            print(f"📈 MTTF: {mttf:.2f} hours")
            print(f"🔒 Reliability in t = {mttf:.2f} hours: {R*100:.2f}%")
            print(f"⚠️ Probability of Failure: {(1 - R)*100:.2f}%")

            # --- VISUALIZATION ---
            fig, ax = plt.subplots(1, 2, figsize=(14, 5))

            # Histogram & PDF
            t_range = np.linspace(min(ttf_data) * 0.8, max(ttf_data) * 1.2, 1000)
            ax[0].hist(ttf_data, bins='auto', density=True, alpha=0.6, label='Data TTF')
            ax[0].plot(t_range, pdf_func(t_range), 'r-', lw=2, label=f'{best_dist} PDF')
            ax[0].set_title('TTF Histogram + PDF Distribution')
            ax[0].set_xlabel('Time (Hours)')
            ax[0].set_ylabel('Density')
            ax[0].legend()
            ax[0].grid(True)

            # Reliability function graph
            ax[1].plot(t_range, rel_func(t_range), 'b-', lw=2, label=f'{best_dist} Reliability')
            ax[1].axvline(mttf, color='green', linestyle='--', label=f'MTTF ≈ {mttf:.0f} jam')
            ax[1].axhline(R, color='orange', linestyle='--', label=f'Reliability @MTTF: {R*100:.1f}%')
            ax[1].set_title('Reliability Function R(t)')
            ax[1].set_xlabel('Time (Hours)')
            ax[1].set_ylabel('Reliability')
            ax[1].set_ylim(0, 1.05)
            ax[1].legend()
            ax[1].grid(True)

            plt.tight_layout()
            plt.show()

        except Exception as e:
            print("❌ An error occurred:", e)

# Event handler
run_button.on_click(on_button_click)

# Display interface
display(input_ttf, run_button, output)


Textarea(value='3107.5\n7677.25\n925\n2751\n8567.67\n7156\n4390.5\n5422.5', description='Data TTF:', layout=La…

Button(button_style='success', description='🔍 Calculate MTTF & Reliability', style=ButtonStyle())

Output()